In [1]:
import string

def check_password_strength(password):
    score = 0

    # Check length
    if len(password) >= 8:
        score += 1
    if len(password) >= 12:
        score += 1

    # Check variety of characters
    if any(c.islower() for c in password):  # lowercase
        score += 1
    if any(c.isupper() for c in password):  # uppercase
        score += 1
    if any(c.isdigit() for c in password):  # numbers
        score += 1
    if any(c in string.punctuation for c in password):  # symbols
        score += 1

    # Simple feedback
    if score <= 2:
        strength = "Weak"
    elif score <= 4:
        strength = "Good"
    else:
        strength = "Strong"

    print(f"Password: {password}")
    print(f"Strength: {strength} (Score: {score}/6)\n")

# Example
check_password_strength("Pass123")
check_password_strength("MyP@ssw0rd")


Password: Pass123
Strength: Good (Score: 3/6)

Password: MyP@ssw0rd
Strength: Strong (Score: 5/6)



In [2]:
import hashlib
import bcrypt

def show_hashes(password):
    md5_hash = hashlib.md5(password.encode()).hexdigest()
    sha_hash = hashlib.sha256(password.encode()).hexdigest()
    bcrypt_hash = bcrypt.hashpw(password.encode(), bcrypt.gensalt())

    print("MD5:", md5_hash)
    print("SHA-256:", sha_hash)
    print("bcrypt:", bcrypt_hash.decode())

# Example
show_hashes("MyPassword123")


MD5: 973d98ac221d7e433fd7c417aa41027a
SHA-256: bc7b8851671f2fda237a53f5057a0376037b6d062e65f965c62aa1d047498759
bcrypt: $2b$12$iG0AzUJ.nz.xaKtTU0iwleW0gZ2A9jA9R/oUR.LCMdeo.1V.RzpPS


In [3]:
import hashlib
import os

def hash_with_salt(password):
    salt = os.urandom(16)  # random salt
    salted_password = salt + password.encode()
    return hashlib.sha256(salted_password).hexdigest()

password = "user123password"

# Without salt
hash1 = hashlib.sha256(password.encode()).hexdigest()
hash2 = hashlib.sha256(password.encode()).hexdigest()

# With salt
salt_hash1 = hash_with_salt(password)
salt_hash2 = hash_with_salt(password)

print("No salt (same each time):")
print(hash1)
print(hash2)
print("\nWith salt (different each time):")
print(salt_hash1)
print(salt_hash2)


No salt (same each time):
6384e02d9a47ac61c64950915c878ea54cc91a3d9c11d7497818017db8356d2a
6384e02d9a47ac61c64950915c878ea54cc91a3d9c11d7497818017db8356d2a

With salt (different each time):
7eddb8f87eb07142cfd7978c109ab9eec782b7b82609dd4043b138137c7d3aad
6879fe3f55c352b7c39b2e5dbbdc6d7a3aa8862aaf0ad8c853ea5e0ae39a079d


In [4]:
import pyotp
import qrcode

# Create a secret for a user
secret = pyotp.random_base32()
totp = pyotp.TOTP(secret)

print("Your secret key:", secret)
print("Current TOTP code:", totp.now())

# Generate QR code (scan with Google Authenticator)
uri = totp.provisioning_uri(name="alice@example.com", issuer_name="MyApp")
qrcode.make(uri).save("totp_qr.png")
print("QR code saved as 'totp_qr.png'")


Your secret key: 3WOLN5MPMH2SUJTHSS3EMJZYVZNMOZ6X
Current TOTP code: 046663
QR code saved as 'totp_qr.png'


In [5]:
import hashlib
import time

common_passwords = ["password", "123456", "letmein", "qwerty", "admin"]

def brute_force(target_hash):
    start = time.time()
    for word in common_passwords:
        guess_hash = hashlib.md5(word.encode()).hexdigest()
        if guess_hash == target_hash:
            print(f"Password found: {word} in {time.time() - start:.4f}s")
            return
    print("Password not found.")

# Example: Hash of "password"
target = hashlib.md5("password".encode()).hexdigest()
brute_force(target)


Password found: password in 0.0000s


In [1]:
import bcrypt
import pyotp

users = {}  # pretend database

def register(username, password):
    # Hash password
    hashed = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
    # Create TOTP secret
    secret = pyotp.random_base32()
    users[username] = {"hash": hashed, "secret": secret}
    print(f"User '{username}' registered successfully!")
    print("TOTP secret:", secret)
    print()

def login(username, password, code):
    if username not in users:
        print("User not found!")
        return

    user = users[username]
    # Check password
    if not bcrypt.checkpw(password.encode(), user["hash"]):
        print("Wrong password!")
        return

    # Check TOTP code
    totp = pyotp.TOTP(user["secret"])
    if totp.verify(code):
        print(" Login successful!")
    else:
        print(" Invalid 2FA code!")

# Example usage
register("alice", "MyStrongP@ss")
secret = users["alice"]["secret"]
code = pyotp.TOTP(secret).now()
login("alice", "MyStrongP@ss", code)


User 'alice' registered successfully!
TOTP secret: LMEGBZMCBUB3BK4G7BG35IYMXTBTHOMX

 Login successful!
